# Colab: Split MNIST 无 Replay 实验

在 Colab 上运行 exp1~exp4，每完成一个实验自动推送到 GitHub。

**前置准备**：
1. 在 [GitHub Settings → Developer settings → Personal access tokens](https://github.com/settings/tokens) 创建 Token，勾选 `repo` 权限
2. 在下方 `GITHUB_TOKEN` 填入你的 Token（或使用 Colab Secrets）

In [ ]:
# ===== 1. 克隆项目 + 安装依赖 =====
import os
import subprocess
import sys

REPO_URL = "https://github.com/bochendong/IdeasLab.git"
PROJECT_DIR = "/content/IdeasLab"

# 克隆（若已存在则 pull）
if os.path.exists(PROJECT_DIR):
    !cd {PROJECT_DIR} && git pull
else:
    !git clone {REPO_URL} {PROJECT_DIR}

os.chdir(PROJECT_DIR)
sys.path.insert(0, PROJECT_DIR)
print("Project dir:", os.getcwd())

In [ ]:
# ===== 2. 安装依赖 =====
!pip install -q torch torchvision
print("PyTorch:", __import__('torch').__version__)

In [ ]:
# ===== 3. 配置 Git 推送（填入你的 Token）=====
# 方式 A: 直接填入（注意：不要在公开分享的 notebook 中暴露）
GITHUB_TOKEN = ""  # 例如: ghp_xxxxxxxxxx

# 方式 B: 使用 Colab Secrets（推荐）
# 在 Colab 左侧钥匙图标中添加 GITHUB_TOKEN，然后取消下行注释：
# from google.colab import userdata
# GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

if not GITHUB_TOKEN:
    from getpass import getpass
    GITHUB_TOKEN = getpass("请输入 GitHub Personal Access Token: ")

# 配置 git 使用 token 推送
push_url = REPO_URL.replace("https://github.com/", f"https://{GITHUB_TOKEN}@github.com/")
!cd {PROJECT_DIR} && git remote set-url origin {push_url}
!cd {PROJECT_DIR} && git config user.email "colab@example.com" 2>/dev/null || true
!cd {PROJECT_DIR} && git config user.name "Colab" 2>/dev/null || true
print("Git 已配置，可推送")

In [ ]:
def run_exp_and_push(script_name: str, run_name: str):
    """运行单个实验，完成后 commit + push"""
    import subprocess
    
    cmd = [sys.executable, os.path.join(PROJECT_DIR, script_name)]
    print(f"\n{'='*60}\nRunning: {script_name}\n{'='*60}")
    ret = subprocess.run(cmd, cwd=PROJECT_DIR)
    if ret.returncode != 0:
        print(f"[ERROR] {script_name} 失败")
        return False
    
    # 提交并推送
    subprocess.run(["git", "add", "output/", "README.md"], cwd=PROJECT_DIR, check=False)
    status = subprocess.run(["git", "status", "--porcelain"], cwd=PROJECT_DIR, capture_output=True, text=True)
    if status.stdout.strip():
        subprocess.run(["git", "commit", "-m", f"[Colab] {run_name} 完成"], cwd=PROJECT_DIR, check=False)
        for branch in ["main", "master"]:
            r = subprocess.run(["git", "push", "origin", branch], cwd=PROJECT_DIR, capture_output=True)
            if r.returncode == 0:
                print(f"[OK] 已推送到 {branch}: {run_name}")
                break
        else:
            print(f"[WARN] push 失败，请检查分支名")
    else:
        print(f"[OK] 无新变更，跳过 push: {run_name}")
    return True

In [ ]:
# ===== 4. 运行实验（每完成一个自动推送）=====
EXPERIMENTS = [
    ("split_mnist/exp2_stronger_reg.py", "Exp1: Stronger Reg"),
    ("split_mnist/exp3_frozen_backbone.py", "Exp2: Frozen Backbone"),
    ("split_mnist/exp4_ewc.py", "Exp3: EWC"),
    ("split_mnist/exp5_frozen_stronger_reg.py", "Exp4: Frozen+Stronger Reg"),
]

for script, name in EXPERIMENTS:
    run_exp_and_push(script, name)

print("\n全部完成！")